### FrozenLake
The FrozenLake environment consists of a 4x4 grid of blocks, each one either being the start block, the goal block, a safe frozen block, or a dangerous hole. The objective is to have an agent learn to navigate from the start to the goal without moving onto a hole. At any given time the agent can choose to move either up, down, left, or right. 

The surface can be described using a grid as follows:<br>
SFFF        
FHFH       
FFFH    
HFFG  

where;<br>
S: starting point, safe<br>
F: frozen surface, safe<br>
H: hole, fall to your doom<br>
G: goal, where the frisbee is located<br>

The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.


### Importing necessary libraries

In [1]:
import gym
import numpy as np
import random
import time

### Setting and exploring environment

In [2]:
# Set up the required environment from OpenAI gym
env = gym.make('FrozenLake-v0')

In [3]:
# Q table will have 's: number of states' rows and 'a: number of actions possible per state' columns
action_space_size = env.action_space.n
state_space_size = env.observation_space.n
print(action_space_size)
print(state_space_size)

4
16


### Initializing Q table 

In [4]:
Q_table = np.zeros([state_space_size, action_space_size])
print(Q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


### Setting necessary parameters

In [5]:
# Set learning parameters
num_episodes = 10000
max_steps_per_episode = 1000
learning_rate = 0.1
discount_rate = 0.99  # gamma

# Parameters for epsilon greedy strategy (Exploration vs Exploitation)
eps = 1             # exploration rate (initial)
max_eps = 1
min_eps = 0.01      # our agent will minimum explore 0.01% of the time
decay_rate = 0.001  # for reducing the exploration rate according to a particular formula after every step to
                    # maintain a exploration exploitation trade off balance 

### Q Learning Algorithm 

In [7]:
rewards_all_episodes = []   
# List to store the reward after each episode
# In this particular environment, the possible options are 0 or 1.
# Reward 0: when the maximum number of steps per episode is reached
# Reward 1: when the agent reaches the goal

for episode in range(num_episodes):  # loop to iterate over each episode
    state = env.reset()         # We need to initialize the environment after every "game over" i.e. for every new episode
    done = False                # Boolean which keeps track of whether our episode is finished
    reward_current_episode = 0  # variable which stores rewards for all time steps for a given episode. 
                                # Initialized to zero after every episode
    
    for step in range(max_steps_per_episode):      # loop to iterate over each time step on an episode
        exploration_rate_threshold = random.uniform(0, 1)    
        # generating a random number to decide whether we should exploit or explore according to epsilon greedy strategy 
        
        if exploration_rate_threshold > eps and ~np.all(Q_table[state,:]==0):
            action = np.argmax(Q_table[state,:])   
        # Exploit the previous updated Q values, choose action whose index has max Q value 
        # Condition 1: value of randomnly generated number > value of epsilon
        # Condition 2: If the table has all 0's for that particular state then donot exploit. 
        # Since using argmax in this case will result in index 0 which will always result in same action
        # The possibility of Q table with all 0's for a particular state is high since we only get a reward +1 on 
        # reaching the final goal thus we will mostly have 0's in our table at the start.
        else:
            action = env.action_space.sample()     # explore by choosing any random action from the action sample space
                                    # gives a number amongst 0, 1, 2, 3 which represent one of the four actions available
         
        # we pass the chosen action to the step function which returns a tuple of values as mentioned
        new_state, reward, done, info = env.step(action)
        
        # updating Q table according to the bellman equation
        Q_table[state, action] = (1-learning_rate)*Q_table[state, action] + learning_rate*(reward + discount_rate*np.max(Q_table[new_state,:]))
        
        state = new_state
        reward_current_episode += reward
                                            
        if done == True:
            break
            
    # decay the value of epsilon according to the decay rate and episode number
    eps = min_eps + (max_eps - min_eps)*np.exp(-decay_rate*episode)    
    rewards_all_episodes.append(reward_current_episode)  
                             
# calculate and print average reward per thousand episodes  
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)                                                                                      
count = 1000
print('Average rewards per thousand episodes')  

for r in rewards_per_thousand_episodes:
    print(count,':',str(sum(r/1000)))
    count+=1000
                                                                                           
# print updated Q table
print('Updated Q table')  
print(Q_table)

Average rewards per thousand episodes
1000 : 0.04500000000000003
2000 : 0.19800000000000015
3000 : 0.3890000000000003
4000 : 0.5620000000000004
5000 : 0.6570000000000005
6000 : 0.6700000000000005
7000 : 0.6640000000000005
8000 : 0.6920000000000005
9000 : 0.6690000000000005
10000 : 0.7010000000000005
Updated Q table
[[0.50544061 0.49650781 0.49687419 0.49699122]
 [0.14509375 0.34801045 0.35005727 0.47942403]
 [0.38653089 0.42589765 0.39700527 0.46156145]
 [0.3569314  0.28352533 0.32773395 0.45423152]
 [0.52022238 0.35863724 0.44948607 0.3332285 ]
 [0.         0.         0.         0.        ]
 [0.40383758 0.07085068 0.15148354 0.09240319]
 [0.         0.         0.         0.        ]
 [0.41383691 0.26635516 0.45482952 0.56639525]
 [0.52896883 0.59930682 0.51164664 0.38610451]
 [0.60342379 0.37126622 0.43208993 0.33421483]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.40464984 0.6136393  0.69698531 0.46495505]
 [0.7409953  0.84565098 0

#### Interpreting the above result:
We know that we only receive a reward of 1 when we reach the goal. So the reward value for first 1000 episodes indicate that our agent reached the goal only 4.5% of the time. But by the end of 10000 episodes we see that the agent learned to reach the goal 70.0% of the time.